<a href="https://colab.research.google.com/github/khatkarprateek/PY580ML_project/blob/main/MatterGenNext200Structures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================================================
# MATTERGEN CRYSTAL STRUCTURE GENERATION - Ba-Ti-O SYSTEM
# ============================================================================
# This notebook generates stable crystal structures for the Ba-Ti-O chemical
# system using Microsoft's MatterGen diffusion model.
#
# Project: MS508 Group Project
# Chemical System: Ba-Ti-O (Barium-Titanium-Oxygen)
# Target: Structures with energy_above_hull ≤ 0.05 eV/atom (thermodynamically stable)
# ============================================================================

In [2]:
# ============================================================================
# NOTES:
# ============================================================================
#
# DEPENDENCY MANAGEMENT:
# - MatterGen officially requires PyTorch 2.2.1+cu118, but we use 2.5.1+cu121
#   for better compatibility with Colab's GPU environment
# - Some version conflicts are expected but don't affect core functionality
# - numpy was downgraded to <2.0 as required by MatterGen
# - monty kept at 2024.7.30 as specified by MatterGen (pymatgen conflict OK)
#
# GENERATION PARAMETERS:
# 1. Energy above hull: Measures thermodynamic stability relative to competing
#    phases. Values ≤0.05 eV/atom indicate structures likely to be synthesizable
#    under normal conditions.
#
# 2. Diffusion guidance factor: Controls the trade-off between structure quality
#    and diversity. Higher values (3.0) produce more physically realistic and
#    stable structures but with less diversity in the generated ensemble.
#
# 3. Ba-Ti-O system: This chemical space includes barium titanate (BaTiO3) and
#    related perovskite structures known for ferroelectric and piezoelectric
#    properties. These materials are widely used in capacitors, sensors, and
#    actuators.
#
# OUTPUT FORMAT:
# - Generated structures are saved as CIF (Crystallographic Information File)
#   files in the specified output directory
# - Each CIF contains atomic positions, lattice parameters, and space group info
# - Structures can be analyzed using:
#   * ASE (Atomic Simulation Environment) - Python-based analysis
#   * pymatgen - Materials analysis and property prediction
#   * VESTA - 3D visualization software
#   * Materials Project database - Compare with known structures
#
# TROUBLESHOOTING:
# - If generation fails, check GPU memory with: !nvidia-smi
# - Reduce batch_size if CUDA out-of-memory errors occur
# - Dependency warnings are normal and usually don't affect functionality
# ============================================================================

In [3]:
# ----------------------------------------------------------------------------
# SECTION 1: Install Git LFS and Clone MatterGen
# ----------------------------------------------------------------------------
# Set up Git LFS for downloading large model files first
!apt-get -y -qq install git-lfs
!git lfs install

# Clone MatterGen repository
!git clone https://github.com/microsoft/mattergen.git
%cd mattergen

Git LFS initialized.
Cloning into 'mattergen'...
remote: Enumerating objects: 1663, done.
remote: Counting objects: 100% (410/410), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 1663 (delta 305), reused 248 (delta 247), pack-reused 1253 (from 2)
Receiving objects: 100% (1663/1663), 3.38 MiB | 33.93 MiB/s, done.
Resolving deltas: 100% (659/659), done.
/content/mattergen


In [4]:
# ----------------------------------------------------------------------------
# SECTION 2: Install PyTorch with CUDA Support
# ----------------------------------------------------------------------------
# Install PyTorch 2.5.1 with CUDA 12.1 support for GPU acceleration
# Note: MatterGen officially requires 2.2.1+cu118, but 2.5.1+cu121 works
print("Installing PyTorch 2.5.1 with CUDA 12.1 support...")



#!pip install torch==2.2.0+cu118 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

!pip install -q --index-url https://download.pytorch.org/whl/cu121 \
  torch==2.5.1+cu121 torchvision torchaudio

!pip install -q torch_geometric

!pip install -q torch-scatter torch-sparse torch-cluster \
  -f https://data.pyg.org/whl/torch-2.5.1+cu121.html

# Verify PyTorch installation and CUDA availability
import torch
import platform

print("\n=== Environment Check ===")
print("Python version:", platform.python_version())
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("========================\n")

Installing PyTorch 2.5.1 with CUDA 12.1 support...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 108.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 140.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB

In [5]:
# ----------------------------------------------------------------------------
# SECTION 3: Install MatterGen Core Dependencies
# ----------------------------------------------------------------------------
print("Installing MatterGen core dependencies...")

# Install configuration management tools
!pip install -q fire hydra-core==1.3.1 hydra-joblib-launcher==1.1.5

# Install deep learning and scientific computing libraries
!pip install -q einops transformers accelerate safetensors tqdm requests contextlib2 lmdb SMACT

# Install materials science packages with MatterGen-compatible versions
!pip install -q "ase==3.25.0" "matplotlib==3.8.4" "monty==2024.7.30"

# Downgrade numpy to satisfy MatterGen requirements
!pip install -q "numpy<2.0"

# Install PyTorch Lightning
!pip install -q pytorch-lightning==2.0.6

Installing MatterGen core dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.4/299.4 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 113.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 6.6 MB/s eta 0:00:00
  

In [6]:
# ----------------------------------------------------------------------------
# SECTION 3: Install MatterGen Core Dependencies
# ----------------------------------------------------------------------------
print("Installing MatterGen core dependencies...")

# Install configuration management tools
!pip install -q fire hydra-core==1.3.1 hydra-joblib-launcher==1.1.5

# Install deep learning and scientific computing libraries
!pip install -q einops transformers accelerate safetensors tqdm requests contextlib2 lmdb SMACT

# Install materials science packages with MatterGen-compatible versions
!pip install -q "ase==3.25.0" "matplotlib==3.8.4" "monty==2024.7.30"

# Downgrade numpy to satisfy MatterGen requirements
!pip install -q "numpy<2.0"

# Install PyTorch Lightning
!pip install -q pytorch-lightning==2.0.6

Installing MatterGen core dependencies...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymatgen 2025.10.7 requires monty>=2025.1.9, but you have monty 2024.7.30 which is incompatible.


In [7]:
# ----------------------------------------------------------------------------
# SECTION 5: Install Additional MatterGen Dependencies
# ----------------------------------------------------------------------------
print("Installing additional MatterGen requirements...")

# Install materials science and development tools
!pip install -q matscipy mattersim autopep8 pylint emmet-core

Installing additional MatterGen requirements...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.1/453.1 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.6/722.6 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.4/536.4 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.8/306.8 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.1/450.1 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 141.9 MB/s 

In [8]:
# ----------------------------------------------------------------------------
# SECTION 6: Install MatterGen Package
# ----------------------------------------------------------------------------
print("Installing MatterGen package...")

#!pip install torch==2.2.0+cu118 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# Core PyG metapackage
!pip install -q torch_geometric

# CUDA ops matching your current Torch (2.8.0+cu126)

# Install MatterGen without strict dependency checking
# (we've already installed compatible versions manually)
!pip install -q --no-deps -e .

Installing MatterGen package...
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for mattergen (pyproject.toml) ... done


In [9]:
# ----------------------------------------------------------------------------
# SECTION 7: Download Pre-trained Model Checkpoint
# ----------------------------------------------------------------------------
print("Downloading pre-trained model checkpoint...")

!git lfs pull -I checkpoints/mattergen_base --exclude=""

# Verify checkpoint exists
!ls -lh checkpoints/mattergen_base/ 2>/dev/null || echo "Checkpoint directory not found"

print("\n=== Installation Complete ===\n")

total 12K
drwxr-xr-x 2 root root 4.0K Dec 10 06:36 checkpoints
-rw-r--r-- 1 root root 5.4K Dec 10 06:36 config.yaml

=== Installation Complete ===



In [10]:
# Quick fix: Force numpy downgrade
!pip install --force-reinstall -q "numpy<2.0"
print("Numpy downgraded successfully")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mattergen 1.0.3 requires jupyterlab>=4.2.5, which is not installed.
mattersim 1.2.0 requires numpy>=2.0.0; python_version >= "3.10", but you have numpy 1.26.4 which is incompatible.
matscipy 1.2.0 requires numpy>=2.0.0, but you have numpy 1.26.4 which is incompatible.
mattergen 1.0.3 requires ase<=3.25.0, but you have ase 3.26.0 which is incompatible.
mattergen 1.0.3 requires monty==2024.7.30, but you have monty 2025.3.3 which is incompatible.
mattergen 1.0.3 requires notebook>=7.2.2, but you have notebook 6.5.7 which is incompatible.
mattergen 1.0.3 requires torch==2.2.1+cu118; sys_platform == "linux", but you have torch 2.5.1+cu121 which is incompatible.
mattergen 1.0.3 requires torchaudio==2.2.1+cu118; sys_platform == "linux", but you have torchaudio 2.5.1+cu121 which is incompatible.
mattergen 1.0.3 requires t

In [11]:
# ----------------------------------------------------------------------------
# SECTION 8: Generate Ba-Ti-O Crystal Structures
# ----------------------------------------------------------------------------
import os

# Use a simple local directory (change this path if needed)
output_path = './ba_ti_o_results'

# Create output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

print(f"Output directory: {output_path}")
print("\n=== Starting Structure Generation ===")
print("Chemical system: Ba-Ti-O (note: MatterGen sorts to Ba-O-Ti internally)")
print("Energy above hull: ≤0.05 eV/atom")
print("Batch size: 100 structures")
print("Diffusion guidance factor: 3.0")
print("=====================================\n")

# Generate crystal structures with specified conditions:
# - Chemical system: Ba-O-Ti (sorted alphabetically as MatterGen requires)
# - Energy above hull: ≤0.05 eV/atom (ensures thermodynamic stability)
# - Diffusion guidance factor: 3.0 (controls generation quality vs diversity)
# - Batch size: 100 structures per batch
# - Number of batches: 1 (total of 100 structures)

!mattergen-generate ./ba_ti_o_results \
  --pretrained-name chemical_system_energy_above_hull \
  --properties_to_condition_on "{'chemical_system': 'Ca-O-Ti-Si', 'energy_above_hull': 0.05}" \
  --diffusion_guidance_factor 3.0 \
  --batch_size 1 \
  --num_batches 100

print("\n=== Generation Complete ===")
print(f"Results saved to: {output_path}")

# List generated files
print("\n=== Generated Files ===")
!ls -lh ./ba_ti_o_results/

# Show first few lines of a sample CIF file if available
print("\n=== Sample Structure (first 20 lines) ===")
!find ./ba_ti_o_results -name "*.cif" | head -1 | xargs head -20

print("\n=== Generation Complete ===")
print(f"Results saved to: {output_path}")
print("Check the output directory for generated CIF files.")

Output directory: ./ba_ti_o_results

=== Starting Structure Generation ===
Chemical system: Ba-Ti-O (note: MatterGen sorts to Ba-O-Ti internally)
Energy above hull: ≤0.05 eV/atom
Batch size: 100 structures
Diffusion guidance factor: 3.0

MODELS_PROJECT_ROOT: /content/mattergen/mattergen
checkpoints/chemical_system_energy_above(…): 100% 563M/563M [00:03<00:00, 182MB/s]
config.yaml: 8.39kB [00:00, 22.3MB/s]

Model config:
/content/mattergen/mattergen/common/utils/data_classes.py:96: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize_config_dir(str(self.model_path)):
adapter:
  adapter:
    _target_: mattergen.adapter.GemNetTAdapter
    atom_type_diffusion: mask
    denoise_atom_types: true
    gemnet:
      _target_: mattergen.common.gemnet.gemnet_ctrl.GemNetTCtrl
      atom_embedding:
        _target_: mattergen.common.gemnet.layers.embedding_block.AtomEmbedding
        